# CUDACha20
## Abstract
The ChaCha20 cipher is very easy to think about in terms of parallelizing it. The keystream is generated in 64-byte blocks, and generating the keystream is done via a counter-mode-esque construction of the stream cipher. This means that the generation is not dependent on any past state (i.e. you can manually set the counter for a block, then consume that state to get a block of the keystream). This allows for easy parallelization where every thread can generate a chunk of blocks of the keystream and individually encrypt sections of the message.

On the security of the stream, the cipher works a little like a pseudorandom function; hence, changing the counter of the state by just one bit result in an entirely different block output. This block output is also assumed to look indistinguishable from a randomly sampled byte string. When the block output is xor'd with the message, what you get is indistinguishable from a randomly sampled byte string.

This cipher is typically used in network communication which is why it is a "stream" cipher because it does not require the message to be padded to exactly the size of the block. It allows for encryption of streams of data where the exact size of the message is not known.

## Specifications
CPU: AMD Ryzen 7 5800x 8 Cores 16 Threads @ 4.8 GHz

GPU: NVIDIA Geforce RTX 3060 Ti

Flags: -O3, --use-fast-math (for GPU optimization since floats are not needed)

Tests are all ran 5 times and averaged.

In [2]:
import subprocess
times = 5

## Performance Testing

### Single-threaded Implementation
File is read/written in 64 KB increments.

In [3]:
avgSingle = 0.0
for _ in range(times):
    p = subprocess.Popen("./cha -d -i 1GB.bin", stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    elapsed = float(str(output.decode()))
    avgSingle += elapsed
avgSingle = avgSingle/times
print("{:.4f} secs {} thread".format(avgSingle, 1))

1.2056 secs 1 thread


I cannot run "perf stat" over a Jupyter Notebook so I will instead provide the text for it.

    Performance counter stats for './cha -i 1GB.bin':

          1,195.14 msec task-clock                       #    1.000 CPUs utilized             
                 4      context-switches                 #    3.347 /sec                      
                 1      cpu-migrations                   #    0.837 /sec                      
               165      page-faults                      #  138.059 /sec                      
     5,736,082,489      cycles                           #    4.799 GHz                         (83.27%)
         1,006,773      stalled-cycles-frontend          #    0.02% frontend cycles idle        (83.27%)
        71,427,946      stalled-cycles-backend           #    1.25% backend cycles idle         (83.27%)
    18,480,310,601      instructions                     #    3.22  insn per cycle            
                                                  #    0.00  stalled cycles per insn     (83.30%)
       215,882,816      branches                         #  180.634 M/sec                       (83.60%)
         2,334,128      branch-misses                    #    1.08% of all branches             (83.30%)

       1.195459966 seconds time elapsed

       1.147260000 seconds user
       0.048136000 seconds sys

The file I encrypted is exactly 1,000,000,000 bytes which makes the cipher run at about 5.74 cycles/byte for a single thread implementation on my machine. This is pretty good by itself, although it can run much faster with specialized CPU instructions.

Now with writes:

In [5]:
avgSingleWrites = 0.0
for _ in range(times):
    p = subprocess.Popen("./cha -d -i 1GB.bin -o out.bin", stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    elapsed = float(str(output.decode()))
    avgSingleWrites += elapsed
avgSingleWrites = avgSingleWrites/times
print("{:.4f} secs {} thread".format(avgSingleWrites, 1))

2.1492 secs 1 thread


    Performance counter stats for './cha -i 1GB.bin -o out.bin':

          2,242.22 msec task-clock                       #    0.998 CPUs utilized             
                79      context-switches                 #   35.233 /sec                      
                 2      cpu-migrations                   #    0.892 /sec                      
               165      page-faults                      #   73.588 /sec                      
    10,702,693,432      cycles                           #    4.773 GHz                         (83.29%)
        36,861,894      stalled-cycles-frontend          #    0.34% frontend cycles idle        (83.25%)
     1,146,846,065      stalled-cycles-backend           #   10.72% backend cycles idle         (83.48%)
    22,510,475,924      instructions                     #    2.10  insn per cycle            
                                                  #    0.05  stalled cycles per insn     (83.43%)
     1,048,748,510      branches                         #  467.727 M/sec                       (83.29%)
       101,819,946      branch-misses                    #    9.71% of all branches             (83.26%)

       2.247592109 seconds time elapsed

       1.121174000 seconds user
       1.121174000 seconds sys

Writes are capped at the speed at which the storage can write. There is not anything really of note right now.

#### Multi-threaded Implementation
File is read/written in 64 KB increments.

In [4]:
avgThreads = []
for t in range(1, 7):
    avg = 0.0
    for _ in range(times):
        p = subprocess.Popen("./cha -d -i 1GB.bin -t {}".format(2**t), stdout=subprocess.PIPE, shell=True)
        (output, err) = p.communicate()
        p_status = p.wait()
        elapsed = float(str(output.decode()))
        avg += elapsed
    avg = avg/times
    print("{:.4f} secs {} threads".format(avg, 2**t))
    avgThreads.append(avg)

0.7523 secs 2 threads
0.3866 secs 4 threads
0.1996 secs 8 threads
0.1494 secs 16 threads
0.1539 secs 32 threads
0.1554 secs 64 threads


    Performance counter stats for './cha -i 1GB.bin -t 16':

          2,193.48 msec task-clock                       #   14.226 CPUs utilized             
            10,574      context-switches                 #    4.821 K/sec                     
                78      cpu-migrations                   #   35.560 /sec                      
               490      page-faults                      #  223.390 /sec                      
     9,911,978,038      cycles                           #    4.519 GHz                         (81.29%)
        15,054,807      stalled-cycles-frontend          #    0.15% frontend cycles idle        (83.59%)
       319,772,834      stalled-cycles-backend           #    3.23% backend cycles idle         (83.32%)
    18,614,726,939      instructions                     #    1.88  insn per cycle            
                                                  #    0.02  stalled cycles per insn     (84.18%)
       247,398,782      branches                         #  112.788 M/sec                       (84.68%)
         5,804,184      branch-misses                    #    2.35% of all branches             (83.09%)

       0.154187514 seconds time elapsed

       1.996491000 seconds user
       0.183310000 seconds sys

The performance scales almost linearly as the number of threads increase, capping at around 16 threads. Amdahl's law can be seen in action here. The cipher is trivially easy to parallelize with threads allowing it to utilize the maximum extent of my CPU. The parallel parts of file encryption make up a majority of the processing done. The only serial part of the program is the keystream generation at a certain block.

With writes enabled:

In [6]:
avgThreadsWrites = []
for t in range(1, 7):
    avg = 0.0
    for _ in range(times):
        p = subprocess.Popen("./cha -d -i 1GB.bin -o out.bin -t {}".format(2**t), stdout=subprocess.PIPE, shell=True)
        (output, err) = p.communicate()
        p_status = p.wait()
        elapsed = float(str(output.decode()))
        avg += elapsed
    avg = avg/times
    print("{:.4f} secs {} threads".format(avg, 2**t))
    avgThreadsWrites.append(avg)

1.1394 secs 2 threads
0.8012 secs 4 threads
0.9165 secs 8 threads
0.9179 secs 16 threads
0.9409 secs 32 threads
0.9633 secs 64 threads


    Performance counter stats for './cha -i 1GB.bin -o out.bin -t 16':

          2,405.45 msec task-clock                       #    2.486 CPUs utilized             
            14,687      context-switches                 #    6.106 K/sec                     
               702      cpu-migrations                   #  291.838 /sec                      
               491      page-faults                      #  204.120 /sec                      
    10,249,780,036      cycles                           #    4.261 GHz                         (83.73%)
        41,165,361      stalled-cycles-frontend          #    0.40% frontend cycles idle        (83.16%)
       696,573,470      stalled-cycles-backend           #    6.80% backend cycles idle         (83.70%)
    21,724,201,076      instructions                     #    2.12  insn per cycle            
                                                  #    0.03  stalled cycles per insn     (84.99%)
       898,568,706      branches                         #  373.556 M/sec                       (82.31%)
        84,161,079      branch-misses                    #    9.37% of all branches             (82.15%)

       0.967711947 seconds time elapsed

       1.346630000 seconds user
       1.054590000 seconds sys

In [5]:
avgGPU = []
for g in range(0, 9):
    avg = 0.0
    for _ in range(times):
        p = subprocess.Popen("./cha -d -i 1GB.bin -g {}".format(1*(2**g)), stdout=subprocess.PIPE, shell=True)
        (output, err) = p.communicate()
        p_status = p.wait()
        elapsed = float(str(output.decode()))
        avg += elapsed
    avg = avg/times
    print("{:.4f} secs {} MB keystream size".format(avg, 1*(2**g)))
    avgGPU.append(avg)

0.3800 secs 1 MB keystream size
0.3624 secs 2 MB keystream size
0.3867 secs 4 MB keystream size
0.3903 secs 8 MB keystream size
0.4018 secs 16 MB keystream size
0.4056 secs 32 MB keystream size
0.3875 secs 64 MB keystream size
0.4029 secs 128 MB keystream size
0.4483 secs 256 MB keystream size
